# Extension of area sources

In [ ]:
%matplotlib inline
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
sys.path.append('/Users/mpagani/Projects/original/oq-hazardlib/')
sys.path.append('/Users/mpagani/Projects/original/oq-engine/')
from utils_model import read_model, get_area
from shapely.wkt import loads
from openquake.hazardlib.source import SimpleFaultSource
from openquake.hazardlib.const import TRT

In [ ]:
sms = {}

modeln = './../../data/xml/emme_fs_2015.xml'
sms['emme'] = read_model(modeln)

modeln = './../../data/xml/EOS_SEAsia.xml'
sms['sea'] = read_model(modeln)

In [ ]:
def get_model_data(sm):
    extr = {}
    src_types = set()
    trt_types = set()
    flens = []
    for src in sm:
        src_types = src_types | set([type(src)])
        trt_types = trt_types | set([src.tectonic_region_type])
        if isinstance(src, SimpleFaultSource) and src.tectonic_region_type == TRT.ACTIVE_SHALLOW_CRUST:
            flens.append(src.fault_trace.get_length())
    print 'Source types:'
    for srct in list(src_types):
        print ('  {0:s}'.format(srct))
    print 'TRTs:'
    for srct in sorted(list(trt_types)):
        print ('  {0:s}'.format(srct))
    print ('\nTotal number of fault sources: {0:d}'.format(len(flens)))
    return flens

In [ ]:
bins = np.logspace(1, 3, num=20)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 12 
mpl.rcParams['ytick.labelsize'] = 12

cols = ['red', 'green', 'blue', 'orange']
colr = ['white', 'white', 'white', 'white']
lwdt = [4, 3, 2, 1]

for idx, key in enumerate(sms.keys()):

    extn = get_model_data(sms[key])
    print ('Min and Max value of log(length): {0:.2f} {1:.2f}'.format(np.log10(min(extn)), np.log10(max(extn))))
    
    histo, bin_edges = np.histogram(extn, bins=bins)
    hist = histo / float(sum(histo))
    
    plt.bar(bin_edges[0:-1]+np.diff(bin_edges)/2, hist, width=np.diff(bin_edges),
            edgecolor=colr[idx], facecolor=cols[idx], 
            linewidth=lwdt[idx], alpha=0.5,
            label=key)

    #plt.hist(extn, bins=bins, label=key, edgecolor=colr[idx],  
    #         normed=True, facecolor=cols[idx], 
    #         linewidth=lwdt[idx], alpha=0.5) 

plt.xlabel(r'Length, [km]', fontsize=14)
plt.grid(which='both', linestyle='--')
plt.xscale('log')
plt.legend()